In [17]:
import os
import pandas as pd
import numpy as np

user_folder = os.path.expanduser('~')
data_folder = os.path.join(user_folder, 'E:/git/database/Toxic_Comment/blends')
files = os.listdir(data_folder)
files

['lgb_submission.csv',
 'Logistic_regression_with_words_and_char_n-grams.csv',
 'Minimal_LSTM_NB-SVM_baseline_ensemble.csv',
 'one_more_blend.csv',
 'Pooled_GRU_Fasttext.csv',
 'toxic_avenger.csv',
 'who09829_gru.csv']

In [18]:
gru = pd.read_csv(os.path.join(data_folder, files[4])) # PL score 0.9829
lstm_nb_svm = pd.read_csv(os.path.join(data_folder, files[2])) # 0.9811
lr = pd.read_csv(os.path.join(data_folder, files[1])) # 0.9788
lgb = pd.read_csv(os.path.join(data_folder, files[0])) # 0.9785
blend_p = pd.read_csv(os.path.join(data_folder, files[3])) # 0.9850

# ave = pd.read_csv(os.path.join(data_folder, files[5])) # 0.9823


weights = pd.Series([0.9829, 0.9811, 0.9788, 0.9785, 0.9850])
weights = 1/weights

### scaling

In [19]:
# Bojan suggests scaling with min-max to make sure that all the submissions have
# orderings that can be compared. Since our metric is AUC, this is okay and may
# improve performance.

from sklearn.preprocessing import minmax_scale
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
for label in labels:
    print('Scaling {}... Please stand by.'.format(label))
    lgb[label] = minmax_scale(lgb[label])
    gru[label] = minmax_scale(gru[label])
    lr[label] = minmax_scale(lr[label])
    lstm_nb_svm[label] = minmax_scale(lstm_nb_svm[label])
    blend_p[label] = minmax_scale(blend_p[label])


Scaling toxic... Please stand by.
Scaling severe_toxic... Please stand by.
Scaling obscene... Please stand by.
Scaling threat... Please stand by.
Scaling insult... Please stand by.
Scaling identity_hate... Please stand by.


In [20]:
#for label in labels:
#    print(label)
#    print(np.corrcoef([gru[label], lstm_nb_svm[label], lr[label], lgb[label], blend_p[label]]))

In [21]:
import copy
submission = copy.deepcopy(gru)

In [22]:
datasets = [gru, lstm_nb_svm, lr, lgb, blend_p]
datasets_rmid = [df.drop('id', axis=1) for df in datasets]

In [53]:
average = gru.drop('id', axis=1)
init = copy.deepcopy(average)

In [54]:
np.random.seed(42)
for i in range(100):
    
    index = np.random.choice(range(5), size=2, replace=False)
    init = init + 0.5*datasets_rmid[index[0]] + 0.5*datasets_rmid[index[1]]

average = (init - average)/100

In [55]:
average.head(3)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.994860,0.360438,0.984070,0.048041,0.948635,0.305460
1,0.002818,0.000959,0.001393,0.000086,0.001850,0.000599
2,0.007591,0.001184,0.004118,0.000196,0.002698,0.000611


In [56]:
submission.iloc[:,1:] = average

In [57]:
submission.to_csv('myBlend_random_result.csv', index=False)

In [58]:
submission.head(1)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.99486,0.360438,0.98407,0.048041,0.948635,0.30546
